# LangChain Redis Kitchen Sink Example

This notebook demonstrates a comprehensive example that combines RedisVectorStore, RedisCache, and RedisChatMessageHistory to create a powerful, efficient, and context-aware chatbot system.

## Setup and Imports

In [17]:
%pip install -qU langchain-redis langchain-openai wikipedia

Note: you may need to restart the kernel to use updated packages.


In [18]:
# ruff: noqa: T201, I001, E501
from langchain_redis.version import __lib_name__

print(f"langchain-redis version: {__lib_name__}")

langchain-redis version: langchain-redis_v0.0.4


Ensure you have a Redis server running. You can start one using Docker with:

```
docker run -d -p 6379:6379 redis:latest
```

Or install and run Redis locally according to your operating system's instructions.

In [19]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://redis:6379


## Importing Required Libraries

In [20]:
from langchain_redis import RedisVectorStore, RedisCache, RedisChatMessageHistory
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.globals import set_llm_cache
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import wikipedia

### Set OpenAI API key

In [21]:
from getpass import getpass

# Check if OPENAI_API_KEY is already set in the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("OpenAI API key not found in environment variables.")
    openai_api_key = getpass("Please enter your OpenAI API key: ")

    # Set the API key for the current session
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OpenAI API key has been set for this session.")
else:
    print("OpenAI API key found in environment variables.")

OpenAI API key found in environment variables.


OpenAI API key has been set for this session.


## Create an Index with RedisVL

In this section, we'll set up our vector store using RedisVL, which provides a powerful interface for creating and managing vector indexes in Redis. We'll define a schema for our Wikipedia data, create an index using RedisVL

In [22]:
from redis import Redis
from redisvl.index import SearchIndex
from redisvl.schema import IndexSchema
from langchain_redis import RedisConfig

### RedisVL Index Schema

We start by defining a schema for our index. This schema includes:
- A text field for the document content
- A text field for metadata
- A vector field for the document embeddings

The vector field is configured with 1536 dimensions (suitable for OpenAI embeddings), using cosine distance and a FLAT index algorithm.

In [23]:
schema = IndexSchema.from_dict(
    {
        "index": {
            "name": "kitchensink_docs",
            "storage_type": "hash",
            "prefix": "wiki",
        },
        "fields": [
            {"name": "text", "type": "text"},
            {"name": "url", "type": "tag"},
            {"name": "title", "type": "text"},
            {
                "name": "embedding",
                "type": "vector",
                "attrs": {
                    "dims": 1536,
                    "distance_metric": "cosine",
                    "algorithm": "FLAT",
                },
            },
        ],
    }
)

### Creating the RedisVL Index

Using the defined schema, we create a SearchIndex object and use it to create the actual index in Redis. This step sets up the structure that our vector store will use.

In [24]:
# Establish Redis connection and define index
redis_client = Redis.from_url(REDIS_URL)

# Create the index using RedisVL
redisvl_index = SearchIndex(schema, redis_client)
redisvl_index.create(overwrite=True)

### Initializing RedisVectorStore

With the RedisVL index in place, we can now initialize our RedisVectorStore. We use a RedisConfig object to specify the index name and Redis URL, ensuring that our vector store connects to the correct index.

In [25]:
# Initialize RedisVectorStore using the pre-constructed index
config = RedisConfig(
    index_name="kitchensink_docs", redis_url=REDIS_URL, from_existing=True
)
vector_store = RedisVectorStore(OpenAIEmbeddings(), config=config)

00:27:49 redisvl.index.index INFO   Index already exists, not overwriting.


### Other Components

We also initialize other components like RedisCache for LLM caching, ChatOpenAI for our language model, and RedisChatMessageHistory for maintaining conversation history.

In [26]:
# Initialize RedisCache
redis_cache = RedisCache(redis_url=REDIS_URL)
set_llm_cache(redis_cache)

In [27]:
# Initialize ChatOpenAI with caching
llm = OpenAI(temperature=0)

In [28]:
# Initialize RedisChatMessageHistory
message_history = RedisChatMessageHistory("kitchensink_chat", redis_url=REDIS_URL)

## Populate Vector Store with Wikipedia Data

In [29]:
## Populate Vector Store with Wikipedia Data

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


def fetch_wikipedia_content(titles):
    documents = []
    for title in titles:
        try:
            page = wikipedia.page(title)
            doc = Document(
                page_content=page.content, metadata={"title": title, "url": page.url}
            )
            documents.append(doc)
        except wikipedia.exceptions.DisambiguationError as e:
            # Choose the first option from the disambiguation list
            page = wikipedia.page(e.options[0])
            doc = Document(
                page_content=page.content,
                metadata={"title": e.options[0], "url": page.url},
            )
            documents.append(doc)
        except wikipedia.exceptions.PageError:
            print(f"Page not found for {title}")
    return documents


# Fetch some Wikipedia articles
titles = [
    "Artificial Intelligence",
    "Deep Learning",
    "Natural Language Processing",
    "Large Language Models",
    "Robotics",
]
documents = fetch_wikipedia_content(titles)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# Add to vector store
vector_store.add_documents(splits)

print(f"Added {len(splits)} document chunks to the vector store.")

Added 450 document chunks to the vector store.


## Create the retriever

In [30]:
# Create the retriever
retriever = vector_store.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def combine_chat_history_and_question(inputs):
    return f"Chat History: {inputs['chat_history']}\nHuman: {inputs['question']}"


# Update the prompt template to include chat history
prompt_template = """
    You are an AI assistant answering questions based on the provided context and chat history. Be concise and accurate.

    Context: {context}
    {question}
    AI Assistant:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create the RAG chain
rag_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["question"])),
        "question": combine_chat_history_and_question,
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

## Interactive Chat Loop

In [31]:
def get_chat_history(history):
    return "\n".join(
        [f"{msg.type.capitalize()}: {msg.content}" for msg in history.messages[-5:]]
    )


print("Welcome to the AI Assistant! Type 'exit' to end the conversation.")

chat_history = []
while True:
    user_input = input("Human: ")
    if user_input.lower() == "exit":
        break

    # Add user message to history
    message_history.add_user_message(user_input)

    # Get response from RAG chain
    result = rag_chain.invoke(
        {"question": user_input, "chat_history": get_chat_history(message_history)}
    )

    # Add AI message to history
    message_history.add_ai_message(result)

    print(f"AI: {result}")

print("Thank you for using the AI Assistant!")

Welcome to the AI Assistant! Type 'exit' to end the conversation.


Human:  What are the principal tenets of A.I.?


AI: 
The principal tenets of AI include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics. These goals are achieved through the use of various techniques such as search and optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics. AI also draws upon fields such as psychology, linguistics, philosophy, neuroscience, and others. The ultimate goal of AI research is to develop artificial general intelligence, which would be able to solve a wide variety of problems with the same breadth and versatility as human intelligence.


Human:  What is the relationship between AI and robotics?


AI: 
AI and robotics have a close relationship, as AI is often used to control and guide robots in their actions and decision-making processes. Robotics also relies on AI for tasks such as perception, learning, and planning. However, AI and robotics are not synonymous, as AI can exist and be used in other fields and applications outside of robotics.


...

AI: 
The core tenets of AI include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics. These are the traditional goals of AI research and are essential for the development and use of AI technology. Additionally, ethical considerations and the promotion of the wellbeing of individuals and communities are also important factors in the development and implementation of AI systems.


Human:  How does AI influence Robotics and vice versa?


AI: 
AI and robotics have a symbiotic relationship, with advancements in one field often leading to advancements in the other. AI technology is used to enhance the capabilities of robots, allowing them to interact with their environment and make decisions based on their programming. On the other hand, robotics provides a physical platform for AI systems to operate in the real world. As AI continues to evolve, it will likely have a significant impact on the development and use of robotics, and vice versa.


Human:  exit


Thank you for using the AI Assistant!


## Analysis of the Kitchen Sink Example

This example demonstrates the integration of multiple Redis-based components in LangChain:

1. **RedisVectorStore**: Used to store and retrieve document chunks from Wikipedia articles. It enables efficient similarity search for relevant context.

2. **RedisCache**: Implemented to cache LLM responses, potentially speeding up repeated or similar queries.

3. **RedisChatMessageHistory**: Stores the conversation history, allowing the AI to maintain context across multiple interactions.

The combination of these components creates a powerful, context-aware chatbot system with the following features:

- **Efficient Information Retrieval**: The vector store allows quick access to relevant information from a large dataset.
- **Improved Response Time**: Caching helps in reducing API calls for similar or repeated queries.
- **Contextual Understanding**: The chat history enables the AI to reference previous parts of the conversation.
- **Scalability**: Redis as a backend allows this system to handle large amounts of data and high traffic efficiently.

This kitchen sink example showcases how these Redis-based components can work together seamlessly in a real-world application, demonstrating the power and flexibility of the langchain-redis package.

## Cleanup

In [32]:
# Clear vector store
vector_store.index.delete(drop=True)

# Clear cache
redis_cache.clear()

# Clear chat history
message_history.clear()

print("Cleanup completed.")

Cleanup completed.
